# Urban footprinter: a convolution-based approach to extract urban extents from LULC rasters

Administrative boundaries are often defined manually case-by-case and based on subjective judgement. As a consequence, their widespread use in the context of urban and environmental studies can lead to misleading outcomes [1, 2].

The aim of the urban footprinter package is to provide a repeatable convolution-based method to extract the extent of the urban regions.

The study area for this example is a rasterized extract of [the official cadastral survey of the Canton of Zurich, Switzerland](https://are.zh.ch/internet/baudirektion/are/de/geoinformation/Themen/amtliche_vermessung.html).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from matplotlib import cm, colors
from scipy import ndimage as ndi

import urban_footprinter as ufp

In [ ]:
raster_filepath = "data/zurich.tif"

with rio.open(raster_filepath) as src:
    lulc_arr = src.read(1)
    res = src.res[0]  # only square pixels are supported
    # custom colormap to plot the raster
    # https://matplotlib.org/3.1.1/tutorials/colors/colormap-manipulation.html
    num_classes = len(np.unique(lulc_arr))
    terrain_r_cmap = cm.get_cmap("terrain_r", num_classes)
    color_values = terrain_r_cmap(np.linspace(0, 1, num_classes))
    blue = np.array([0, 0.8, 1, 1])
    color_values[13, :] = blue  # water
    color_values[14, :] = blue  # water
    cmap = colors.ListedColormap(color_values)
    # np.where is used because nodata values of 255 would distort the colormap
    plt.imshow(np.where(lulc_arr != 255, lulc_arr, -1), cmap=cmap)
    plt.colorbar()

According to the land cover classification of the dataset, the codes 0 to 7 correspond to urban land cover classes. Therefore, the array of urban/non-urban pixels looks as follows:

In [ ]:
urban_lulc_classes = range(8)
urban_lulc_arr = np.isin(lulc_arr, urban_lulc_classes).astype(np.uint32)
plt.imshow(urban_lulc_arr)

## How the urban footprinter works

The approach of the urban footprinter library is built upon the methods used in the "Atlas of Urban Expansion" by Angel et al. [3]. The main idea is that a pixel is considered part of the urban extent depending on the proportion of built-up pixels that surround it. Accordingly, the extraction of the urban extent can be customized by means of the following three parameters:

* `kernel_radius`: radius (in meters) of the circular kernel used in the convolution
* `urban_threshold`: proportion of neighboring (within the kernel) urban pixels after which a given pixel is considered urban
* `buffer_dist` (optional): buffer distance (in meters) to add around the computed mask

### Convolution with a circular kernel

The main task consists in computing, for each pixel of the raster, the proportion of surrounding build-up pixels. This is done by means of a convolution with a circular kernel. In this example, the `kernel_radius` parameter will be set to 500 meters.

In [ ]:
kernel_radius = 500

In [ ]:
kernel_pixel_radius = int(kernel_radius // res)
kernel_pixel_len = 2 * kernel_pixel_radius + 1

y, x = np.ogrid[
    -kernel_pixel_radius : kernel_pixel_len - kernel_pixel_radius,
    -kernel_pixel_radius : kernel_pixel_len - kernel_pixel_radius,
]
mask = x * x + y * y <= kernel_pixel_radius * kernel_pixel_radius

kernel = np.zeros((kernel_pixel_len, kernel_pixel_len), dtype=np.uint32)
kernel[mask] = 1

The kernel will be an array of ones (the circle) and zeros which looks as follows:

In [ ]:
plt.imshow(kernel)
plt.colorbar()

The convolution is computed by means of the [convolve](https://docs.scipy.org/doc/scipy-0.16.0/reference/ndimage.html) function of SciPy.

In [ ]:
conv_result = ndi.convolve(urban_lulc_arr, kernel)

Since both `urban_lulc_arr` and `kernel` are arrays of ones and zeros, the result corresponds to the number of built-up pixels that lie within a 500m distance.

In [ ]:
plt.imshow(conv_result)
plt.colorbar()

### Filtering the convolution results

The result of the convolution is then used to classify pixels that are part of the urban extent. To this end, the `urban_threshold` parameter sets the proportion of surrounding built-up pixels after which a pixel is to be considered part of the urban extent. In this example, it will be set to 0.25 (i.e., 25%).

In [ ]:
urban_threshold = 0.25

In [ ]:
urban_mask = conv_result >= urban_threshold * np.sum(kernel)

The result looks as follows:

In [ ]:
plt.imshow(urban_mask)

### Optional: extract only the largest urban clusters

The fact that raster images are rectangular and real-world cities are not often entails that the raster includes more than just one urban settlement. In such case, the user might be interested in the extent that corresponds to the main urban settlement, namely, the largest urban patch of the raster. The largest urban cluster of the resulting `urban_mask` can be obtained by means of the [label](https://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.ndimage.measurements.label.html#scipy.ndimage.measurements.label) function of SciPy, which performs a connected-component labeling of the input array (i.e., `urban_mask`).

In [ ]:
num_patches = 2

kernel_moore = ndi.generate_binary_structure(2, 2)
label_arr = ndi.label(urban_mask, kernel_moore)[0]
# get the (pixel) counts of each urban patch label
labels, counts = np.unique(label_arr, return_counts=True)
# sort the urban patch label by (pixel) counts and delete the 0 (which in
# `label_arr` always corresponds to the nodata values given the way `ndi.label`
# works
sorted_labels = labels[np.argsort(-counts)]
sorted_labels = sorted_labels[sorted_labels > 0]
# now let `urban_mask` include only the n-largest urban patches where n is
# `num_patches`
urban_mask = np.isin(label_arr, sorted_labels[:num_patches])

In [ ]:
plt.imshow(urban_mask)

### Optional: adding a buffer around the mask

In some contexts, e.g., the study of the urban-rural gradient in landscape ecology, it might be of interest to add a buffer around the urban extent. In this example, a buffer of 1000 meters will be added to the extracted urban extent.

In [ ]:
buffer_dist = 1000

To this end, the [binary_dilation](https://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.ndimage.morphology.binary_dilation.html#scipy.ndimage.morphology.binary_dilation) function of Scipy can be used as follows:

In [ ]:
iterations = int(buffer_dist // res)
kernel_moore = ndi.generate_binary_structure(2, 2)
urban_mask = ndi.binary_dilation(urban_mask, kernel_moore, iterations=iterations)

In [ ]:
plt.imshow(urban_mask)

## The urban footprinter API

The urban footprinter package provides two ways to execute the procedure described above. On the one hand, an *object-oriented* interface which is well suited for the interactive exploration with different parameter values. On the other hand, the whole procedure is also encapsulated into a single function.

### Object-oriented approach

The `UrbanFootprinter` class can instantiated as follows:

In [ ]:
help(ufp.UrbanFootprinter.__init__)

In [ ]:
urban_classes = list(range(8))

uf = ufp.UrbanFootprinter(raster_filepath, urban_classes=urban_classes)

Then, given the `compute_footprint_mask` will compute the urban extent mask according to the provided `kernel_radius`, `urban_threshold`, and optionally `num_patches` and `buffer_dist` arguments:

In [ ]:
kernel_radius = 500
urban_threshold = 0.25

urban_mask = uf.compute_footprint_mask(kernel_radius, urban_threshold)
plt.imshow(urban_mask)

Note that the execution of `compute_footprint_mask` takes some time, since it is when the convolution (the most expensive task of the workflow) is executed. The main advantage of using the `UrbanFootprinter` class, is that the convolution results are cached, so that further calls avoid performing it if it is not necessary. For instance, imagine that we wanted to set `urban_threshold` to 10%. The call to `compute_footprint_mask` below will compute the urban mask accordingly *without having to perform the convolution again*:

In [ ]:
urban_threshold = 0.10

urban_mask = uf.compute_footprint_mask(kernel_radius, urban_threshold)
plt.imshow(urban_mask)

The same applies if we wanted to add a buffer around the mask:

In [ ]:
urban_mask = uf.compute_footprint_mask(kernel_radius, urban_threshold, buffer_dist=1000)
plt.imshow(urban_mask)

Nevertheless, note that if we wanted to change the radius of the convolution kernel, a new (different) convolution operation would have to be performed.

Finally, the urban extent can also be obtained as a vector geometry (instead of a raster array) by means of the `compute_footprint_mask_shp` method:

In [ ]:
urban_mask = uf.compute_footprint_mask_shp(kernel_radius, urban_threshold)
urban_mask

### Single function approach

If interactive exploration of the parameters is not required, the whole procedure described above can be executed with the single function named `urban_footprint_mask`, which accepts the following arguments:

In [ ]:
help(ufp.urban_footprint_mask)

Therefore, the urban extent extracted above can be obtained as in:

In [ ]:
urban_mask = ufp.urban_footprint_mask(
    raster_filepath, kernel_radius, urban_threshold, urban_classes=urban_classes
)
plt.imshow(urban_mask)

Like in the object-oriented approach, the urban extent can also be obtained as a vector geometry (instead of a raster array) by means of the `urban_footprint_mask_shp` function:

In [ ]:
urban_mask = ufp.urban_footprint_mask_shp(
    raster_filepath, kernel_radius, urban_threshold, urban_classes=urban_classes
)
urban_mask

## References

1. Oliveira, E. A., Andrade Jr, J. S., & Makse, H. A. (2014). Large cities are less green. Scientific reports, 4, 4235.
2. Rozenfeld, H. D., Rybski, D., Andrade, J. S., Batty, M., Stanley, H. E., & Makse, H. A. (2008). Laws of population growth. Proceedings of the National Academy of Sciences, 105(48), 18702-18707.
3. Angel, S., Blei, A. M., Civco, D. L., & Parent, J. (2016). Atlas of urban expansion - The 2016 edition, Volume 1: Areas and Densities. New York: New York University, Nairobi: UN-Habitat, and Cambridge, MA: Lincoln Institute of Land Policy.